In [1]:
import pandas as pd
import numpy as np
import env
import wrangle

In [3]:
df = wrangle.get_zillow_data()

In [4]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,4.0,3.5,3100.0,1023282.0,1998.0,11013.72,6059.0
1,2.0,1.0,1465.0,464000.0,1967.0,5672.48,6111.0
2,3.0,2.0,1243.0,564778.0,1962.0,6488.30,6059.0
3,4.0,3.0,2376.0,145143.0,1970.0,1777.51,6037.0
4,3.0,3.0,1312.0,119407.0,1964.0,1533.89,6037.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77614 entries, 0 to 77613
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bedroomcnt                    77580 non-null  float64
 1   bathroomcnt                   77580 non-null  float64
 2   calculatedfinishedsquarefeet  77379 non-null  float64
 3   taxvaluedollarcnt             77579 non-null  float64
 4   yearbuilt                     77310 non-null  float64
 5   taxamount                     77575 non-null  float64
 6   fips                          77580 non-null  float64
dtypes: float64(7)
memory usage: 4.1 MB


In [10]:
for col in df:
    print(df[col].value_counts(dropna=False))
    print('-----------')
    
# 

3.0     30437
2.0     19223
4.0     17551
5.0      4550
1.0      3391
6.0      1000
0.0       837
8.0       253
7.0       208
9.0        70
NaN        34
10.0       31
12.0       16
11.0        9
13.0        2
16.0        1
14.0        1
Name: bedroomcnt, dtype: int64
-----------
2.0     31576
3.0     17354
1.0     12945
2.5      6607
4.0      3356
1.5      1419
3.5      1036
5.0      1026
4.5       696
0.0       599
6.0       419
5.5       224
7.0       114
8.0       108
6.5        47
NaN        34
9.0        23
7.5        16
10.0        7
8.5         3
11.0        3
13.0        1
18.0        1
Name: bathroomcnt, dtype: int64
-----------
NaN       235
1200.0    182
1120.0    169
1440.0    148
1080.0    133
         ... 
5333.0      1
5178.0      1
5396.0      1
4288.0      1
6758.0      1
Name: calculatedfinishedsquarefeet, Length: 4973, dtype: int64
-----------
400000.0     56
600000.0     50
350000.0     49
450000.0     45
420000.0     44
             ..
124876.0      1
247786.0    

> These are the missing values
> - 34 in `bedroomcnt` 
> - 235 in `calculatedfinishedsquarefeet` 
> - 34 in `bathroomcnt`
> - 39 in `taxamount`
> - 34 in `fips`